# Import All Packages




In [1]:
    import random
    import pandas as pd
    import numpy as np
    import string
    from string import digits
    from sklearn.model_selection import train_test_split
    import torch
    import torch.nn as nn
    from sklearn.metrics import classification_report
    import transformers
    from transformers import AutoModel, BertTokenizerFast
    from ipywidgets import IntProgress
    from tqdm import tqdm



# Import BERT Model, BERT Tokenizer and Torch

In [2]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# import Torch
device = torch.device("cpu")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Predefined CLass

In [6]:
class Propaganda:
    NEGATIVE = 0
    POSITIVE = 1
class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = SUBJprop


class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [int(x.propaganda) for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == str(Propaganda.NEGATIVE), self.reviews))
        positive = list(filter(lambda x: x.propaganda == str(Propaganda.POSITIVE), self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)

# Split train dataset into train, validation and test sets

In [4]:
# step 2: Load Data
reviews = []
data  = pd.read_excel('Data/finalDataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))


print("Total Rows:")
print(len(reviews))
print("Total Positive:")
print(len(list(filter(lambda x: x.propaganda == str(Propaganda.POSITIVE), reviews))))
print("Total Negative:")
print(len(list(filter(lambda x: x.propaganda == str(Propaganda.NEGATIVE), reviews))))

Total Rows:
14058
Total Positive:
3904
Total Negative:
10154


In [5]:
neg_prop = list(filter(lambda x: x.propaganda == str(Propaganda.NEGATIVE), reviews))
pos_prop = list(filter(lambda x: x.propaganda == str(Propaganda.POSITIVE), reviews))
########################################################################################
#split trainig and DevTest dataset
neg_train, neg_devtest  = train_test_split(neg_prop , train_size=0.7, shuffle= False )
pos_train, pos_devtest = train_test_split(pos_prop , train_size=0.7, shuffle= False )
########################################################################################
#prepare training dataset
train = neg_train + pos_train
#random.shuffle(train)
########################################################################################
#prepare development and test dataset
neg_dev, neg_test = train_test_split(neg_devtest , train_size=0.5, shuffle= False )
pos_dev, pos_test = train_test_split(pos_devtest , train_size=0.5, shuffle= False )

dev = neg_dev + pos_dev
#random.shuffle(dev)

test = neg_test + pos_test
#random.shuffle(test)

In [15]:
# step 3: Seperate the attribute, originally our array has text and score. we want them to be a seperate array
train_container = ReviewContainer(train)
train_container.evenly_distribute()

train_text = train_container.get_sentence()   
train_labels = train_container.get_propaganda() 

dev_text = [x.sentence for x in dev]
dev_labels = [int(x.propaganda) for x in dev]

test_text = [x.sentence for x in test]
test_labels = [int(x.propaganda) for x in test]



### Save splited Data to seprate excel files

In [13]:
# Create Excel file of train dataset
train_df = pd.DataFrame(train_text, columns=["Sentence"])
train_df['SUBJprop'] = train_labels
train_df.to_excel("./Data/trainDataset.xlsx")
# Create Excel file of dev dataset
dev_df = pd.DataFrame(dev_text, columns=["Sentence"])
dev_df['SUBJprop'] = dev_labels
dev_df['Tanbih'] = ""
dev_df.to_excel("./Data/devDataset.xlsx")
# Create Excel file of test dataset
test_df = pd.DataFrame(test_text, columns=["Sentence"])
test_df['SUBJprop'] = test_labels
test_df.to_excel("./Data/testDataset.xlsx")

In [9]:
max_seq_len = 20


<class 'int'>


In [16]:

# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    dev_text,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2105: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Convert Integer Sequences to Tensors

In [17]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels)



# for validation set
dev_seq = torch.tensor(tokens_val['input_ids'])
dev_mask = torch.tensor(tokens_val['attention_mask'])
dev_y = torch.tensor(dev_labels)

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels)

print(test_seq)
print(test_mask)
print(test_y)

tensor([[ 101, 2040, 2056,  ...,    0,    0,    0],
        [ 101, 1999, 1996,  ...,  102,    0,    0],
        [ 101, 6854, 1010,  ..., 1996, 6114,  102],
        ...,
        [ 101, 1999, 3218,  ..., 4491, 2000,  102],
        [ 101, 1998, 2057,  ..., 2151, 4013,  102],
        [ 101, 2130, 6715,  ..., 2031, 3622,  102]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
tensor([0, 0, 0,  ..., 1, 1, 1])


# Classifiers

## SVM

In [18]:

from sklearn import svm

clf_svm = svm.SVC(kernel='rbf',C=1, probability=True)
clf_svm.fit(train_seq, train_y)

clf_svm.predict(test_seq)

array([0, 1, 1, ..., 1, 1, 1])

## Decision Tree

In [19]:

from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_seq , train_y)

clf_dec.predict(test_seq)

array([0, 1, 1, ..., 1, 0, 1])

## Naive Bayes

In [20]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_seq , train_y)

clf_gnb.predict(test_seq)

array([0, 0, 0, ..., 1, 0, 1])

## Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_seq, train_y)

clf_log.predict(test_seq)

array([0, 0, 1, ..., 1, 1, 1])

# Evaluation

# F1

In [22]:
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_y, clf_svm.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Support Decision Tree
print(f1_score(test_y,clf_dec.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Support Naive Bayes
print(f1_score(test_y,clf_gnb.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Logistic Regression
print(f1_score(test_y,clf_log.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


[0.39683586 0.66148724]
[0.40754717 0.6418251 ]
[0.37074584 0.69390631]
[0.43512748 0.59389002]


## Mean Accuracy

In [23]:
# For Support Vector Machine
print(clf_svm.score(test_seq,test_y))
# For Decision Tree
print(clf_dec.score(test_seq,test_y))
# For Decision Naive Bayes
print(clf_gnb.score(test_seq,test_y))
# For Logistic Regression
print(clf_log.score(test_seq,test_y))

0.566350710900474
0.5535545023696683
0.5881516587677725
0.5274881516587677


## Precision

In [24]:
# Precision
from sklearn.metrics import precision_score

# For Support Vector Machine
print(precision_score(test_y, clf_svm.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Tree
print(precision_score(test_y, clf_dec.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Naive Bayes
print(precision_score(test_y, clf_gnb.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Logistic Regression
print(precision_score(test_y, clf_log.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

[0.32330827 0.75826972]
[0.32270916 0.76311031]
[0.32201258 0.74904943]
[0.32569975 0.783029  ]


## Recall

In [25]:
from sklearn.metrics import recall_score

# For Support Vector Machine
print(recall_score(test_y, clf_svm.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Tree
print(recall_score(test_y, clf_dec.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Naive Bayes
print(recall_score(test_y, clf_gnb.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Logistic Regression
print(recall_score(test_y, clf_log.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

[0.51365188 0.58661417]
[0.55290102 0.55380577]
[0.43686007 0.64632546]
[0.6552901  0.47834646]


# VIP : Predict_Proba using Threshold

In [26]:
AboveThresholdsvm = [] 
AboveThresholdsvmbrop = []
# [0][1] = Postivie
# [0][0] = Negative


for i in range(len(test_seq)):
  if clf_svm.predict_proba(test_seq[i].reshape(1, -1))[0][1]>0.60 or clf_svm.predict_proba(test_seq[i].reshape(1, -1))[0][0]>0.60:
    AboveThresholdsvm.append(test_text[i])
    AboveThresholdsvmbrop.append(test_labels[i])



print(len(AboveThresholdsvm))


191


In [27]:
AboveThresholdLR = [] 
AboveThresholdLRbrop = []
# [0][1] = Postivie
# [0][0] = Negative


for i in range(len(test_seq)):
  if clf_log.predict_proba(test_seq[i].reshape(1, -1))[0][1]>0.60 or clf_log.predict_proba(test_seq[i].reshape(1, -1))[0][0]>0.60:
    AboveThresholdLR.append(test_text[i])
    AboveThresholdLRbrop.append(test_labels[i])



print(len(AboveThresholdsvm))

191


In [28]:
# Bert
tokens_testsvm = tokenizer.batch_encode_plus(
    AboveThresholdsvm,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
test_seqsvm = torch.tensor(tokens_testsvm['input_ids'])
test_masksvm = torch.tensor(tokens_testsvm['attention_mask'])
test_ysvm = torch.tensor(AboveThresholdsvmbrop)

tokens_testLR = tokenizer.batch_encode_plus(
    AboveThresholdLR,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)


test_seqLR = torch.tensor(tokens_testLR['input_ids'])
test_maskLR = torch.tensor(tokens_testLR['attention_mask'])
test_yLR = torch.tensor(AboveThresholdLRbrop)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2105: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Evaluation

In [29]:
#F1
print("F1 SVM more than 60% threshold")
print(f1_score(test_ysvm, clf_svm.predict(test_seqsvm),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

print("F1 LR more than 60% threshold")
print(f1_score(test_yLR, clf_log.predict(test_seqLR),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

F1 SVM more than 60% threshold
[0.48275862 0.41340782]
F1 LR more than 60% threshold
[0.42827443 0.74607572]


In [30]:
# Accuracy
# For Support Vector Machine
print("Accuracy SVM more than 60% threshold")
print(clf_svm.score(test_seqsvm,test_ysvm))
print("Accuracy LR more than 60% threshold")
print(clf_log.score(test_seqLR,test_yLR))

Accuracy SVM more than 60% threshold
0.450261780104712
Accuracy LR more than 60% threshold
0.6483375959079284


In [31]:
# Precision
from sklearn.metrics import precision_score
# For Support Vector Machine
print("Precision SVM more than 60% threshold")
print(precision_score(test_ysvm, clf_svm.predict(test_seqsvm),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
print("Precision LR more than 60% threshold")
print(precision_score(test_yLR, clf_log.predict(test_seqLR),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


Precision SVM more than 60% threshold
[0.33333333 0.84090909]
Precision LR more than 60% threshold
[0.3639576  0.80961924]


In [32]:
# Recall
from sklearn.metrics import recall_score
print("Precision SVM more than 60% threshold")
print(recall_score(test_ysvm, clf_svm.predict(test_seqsvm),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

print("Precision LR more than 60% threshold")
print(recall_score(test_yLR, clf_log.predict(test_seqLR),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

Precision SVM more than 60% threshold
[0.875      0.27407407]
Precision LR more than 60% threshold
[0.52020202 0.69178082]
